In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

from tensorflow.keras.applications import DenseNet121
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,MaxPooling2D , Conv2D,Flatten
from tensorflow.keras.models import Sequential

from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report


from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

2024-07-23 04:25:41.147020: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 04:25:41.147164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 04:25:41.292983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train_dir = "/kaggle/input/ucf-crime-dataset/Train"
test_dir = "/kaggle/input/ucf-crime-dataset/Test"

SEED = 12
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 64
EPOCHS = 1
LR =  0.00003

In [5]:
train2 = train_dir
test2 = test_dir

In [6]:
train2

'/kaggle/input/ucf-crime-dataset/Train'

# **Implementing a ConvLSTM Network**

In [7]:
# Implementing the ConvLSTM Network
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets

print('Done')

Done


In [ ]:
train_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(train2, transform=transforms.ToTensor()),
        batch_size=16,
        shuffle=True,
    )
print('done')

In [ ]:
# Loads the images from train2 dataset and transforms them into tensors.

test_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(test2, transform=transforms.ToTensor()),
        batch_size=16,
        shuffle=False,
    )

in_channels = 3
hidden_channels = 64
num_classes = 14
    
print('Done')

# Why images need to be converted to Tensors?
# Tensors are understood by PyTorch, tensors are more efficient than NumPy arrays. They're optimized for use with PyTorch.

# **Implementing a Deep Learning Model**

Dataset has already been imported as train_dir and test_dir

In [ ]:
crime_types=os.listdir(train_dir)
n=len(crime_types)
print("Number of crime categories : ",n)

In [ ]:
crimes={}
train=test=0
for clss in crime_types:
    num=len(os.listdir(os.path.join(train_dir,clss)))
    train+=num
    test+=len(os.listdir(os.path.join(test_dir,clss)))
    
    crimes[clss]=num

In [ ]:
plt.figure(figsize=(15, 10))
plt.pie(x=np.array([train,test]), autopct="%.1f%%", explode=[0.1, 0.1], labels=["Training Data", "Test Data"], pctdistance=0.5)
plt.title("Share of train and test images ", fontsize=14);

In [ ]:
plt.figure(figsize=(10, 7))
plt.barh(list(crimes.keys()), list(crimes.values()), height=0.6, align="center")
plt.yticks(rotation=0)

plt.xlabel("Frame count", fontsize=14)
plt.ylabel("Classification", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(15,10))
plt.pie(x=np.array(list(crimes.values())), autopct="%.1f%%", explode=[0.1]*n,labels=list(crimes.keys()), pctdistance=0.5)
plt.title("Share of train and test images ", fontsize=14);

In [ ]:
IMG_WIDTH=64
IMG_HEIGHT=64

IMG_SHAPE=(IMG_HEIGHT,IMG_WIDTH)
seed=69

INPUT_SHAPE=(IMG_HEIGHT,IMG_WIDTH,3)

In [ ]:
train_set=image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SHAPE,
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset="training",
)

val_set=image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    batch_size=BATCH_SIZE,
    image_size=IMG_SHAPE,
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset="validation",
)

test_set=image_dataset_from_directory(
    test_dir,
    label_mode="categorical",
    class_names=None,
    batch_size=BATCH_SIZE,
    image_size=IMG_SHAPE,
    shuffle=False,
    seed=seed,
)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Function to evaluate metrics
def evaluate_model(model, test_set):
    y_true = np.concatenate([y for x, y in test_set], axis=0)
    y_pred_prob = model.predict(test_set)
    y_pred = np.argmax(y_pred_prob, axis=1)
    y_true = np.argmax(y_true, axis=1)

    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    y_true_categorical = to_categorical(y_true, num_classes=y_pred_prob.shape[1])
    auc = roc_auc_score(y_true_categorical, y_pred_prob, average='weighted', multi_class='ovr')

    results = {
        "accuracy": model.evaluate(test_set)[1],
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "auc": auc
    }
    return results

In [ ]:
# Base model function
def base_model():
    return DenseNet121(include_top=False, input_shape=INPUT_SHAPE, weights="imagenet")

# Model 1: GAP + Dropout + Dense Layers
def create_model_1():
    model = Sequential([
        base_model(),
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 2: GAP
def create_model_2():
    model = Sequential([
        base_model(),
        GlobalAveragePooling2D(),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 3: GAP + Dropout
def create_model_3():
    model = Sequential([
        base_model(),
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 4: GAP + Dense Layer
def create_model_4():
    model = Sequential([
        base_model(),
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 5: Dropout + Dense Layer
def create_model_5():
    model = Sequential([
        base_model(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 6: Dropout
def create_model_6():
    model = Sequential([
        base_model(),
        Dropout(0.5),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Model 7: Dense Layers
def create_model_7():
    model = Sequential([
        base_model(),
        Dense(256, activation='relu'),
        Dense(n, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Training and evaluating all models
models = {
    "M1": create_model_1(),
    "M2": create_model_2(),
    "M3": create_model_3(),
    "M4": create_model_4(),
    "M5": create_model_5(),
    "M6": create_model_6(),
    "M7": create_model_7()
}

In [ ]:
results = {}
for name, model in models.items():
    print(f"Training and evaluating {name}...")
    model.fit(train_set, validation_data=val_set, epochs=10)
    results[name] = evaluate_model(model, test_set)

# Display results
import pandas as pd

metrics_df = pd.DataFrame(results).T
metrics_df

In [ ]:
print("done")